In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
files = []
for file in os.listdir('data'):
    if file.endswith('.csv'):
        files.append(os.path.join('data', file))

In [3]:
data = []

for filename in files:
    
    
    with open(filename, 'r') as file:
        ticker = filename.split('_')[-1][:-4]
        line = file.readline()
        line = file.readline()
        while(line):
            ts =line.split(';"')[0][1:].split(';')[0]
            tweet = line.split(';"')[1].split('";')[0]
            line=file.readline()
            data.append([ticker,ts,tweet])
        
    
df = pd.DataFrame(data, columns=['ticker','timestamp','tweet'])
#df

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155011 entries, 0 to 155010
Data columns (total 3 columns):
ticker       155011 non-null object
timestamp    155011 non-null object
tweet        155011 non-null object
dtypes: object(3)
memory usage: 3.5+ MB


In [10]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [30]:
analyzer = SentimentIntensityAnalyzer()
for tweet in df['tweet']:
    vs = analyzer.polarity_scores(tweet)
    vs_val = list(vs.values())
    df['neg'] = vs_val[0]
    df['neu'] = vs_val[1]
    df['pos'] = vs_val[2]
    df['compound'] = vs_val[3]

In [31]:
df.head()

,ticker,timestamp,tweet,neg,neu,pos,compound
0,INTC,2019-03-21 23:29,$ AAPL $ TRV $ MSFT $ HD $ DWDP $ INTC $ NKE $...,0.046,0.954,0.0,-0.3034
1,INTC,2019-03-21 23:25,$ INTC Intel - Q3 2019 Intel Corporation Earni...,0.046,0.954,0.0,-0.3034
2,INTC,2019-03-21 23:25,$ INTC # Intel # Intelligence # ImplementingCl...,0.046,0.954,0.0,-0.3034
3,INTC,2019-03-21 23:25,$ INTC Intel - Q2 2019 Intel Corporation Earni...,0.046,0.954,0.0,-0.3034
4,INTC,2019-03-21 23:24,$ INTC Intel - Q1 2019 Intel Corporation Earni...,0.046,0.954,0.0,-0.3034


In [37]:
df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)

In [47]:
df['day_name'] = df['timestamp'].map(lambda x: x.day_name())
df['day'] = df['timestamp'].map(lambda x: x.day)
df['day_of_week'] = df['timestamp'].map(lambda x: x.dayofweek)

In [53]:
df.to_csv('tweets_with_sentiment.csv')